In [ ]:
# Load the relevant packages.
import numpy as np
import pandas as pd
import scanpy as sc
import matplotlib.pyplot as plt
import seaborn as sns
import anndata
import scvelo as scv
from scipy import sparse

sc.settings.verbosity = 3 # Possible values: (0) errors, (1) warnings, (2) info, (3) hints
sc.settings.set_figure_params(dpi = 100, facecolor='white', fontsize=18, transparent=True)
scv.settings.set_figure_params('scvelo')

In [ ]:
# Load the processed fibroblast data
results_directory = '../data/' # Initialise the results file

In [ ]:
# Load the file
file_name = 'integratedfibroblastsdata.h5ad'
fibroblasts_merged = sc.read_h5ad(results_directory + file_name)

In [ ]:
# Change the sample colours to match the paper
new_sample_colours = ['#3284fc','#393b79', '#c1c1ea', '#6b6ecf','#edca8b', '#c39d15', '#f1919b', '#e75868']
fibroblasts_merged.uns['sample_colors'] = new_sample_colours

# RNA Velocity

Run RNA velocity on a copy of the original scanpy object, as RNA velocity analysis will filter out genes.

In [ ]:
fib_rna = fibroblasts_merged.copy()

In [ ]:
scv.pp.filter_and_normalize(fib_rna, min_shared_counts=20)

We use the integrated latent embedding produced by Scanorama to try and account for the multiple conditions.

In [ ]:
scv.pp.moments(fib_rna, use_rep='X_SC_FIB', n_neighbors=30) 

In [ ]:
scv.tl.recover_dynamics(fib_rna)
scv.tl.velocity(fib_rna, mode='dynamical')

In [ ]:
scv.tl.velocity_graph(fib_rna)

In [ ]:
scv.tl.velocity_pseudotime(fib_rna)

## Subclusters as partitions

In [ ]:
scv.pl.velocity_embedding_stream(fib_rna, basis='umap', color='leiden_sub', legend_fontoutline=2.5)

In [ ]:
# this is needed due to a current bug - bugfix is coming soon.
fib_rna.uns['neighbors']['distances'] = fib_rna.obsp['distances']
fib_rna.uns['neighbors']['connectivities'] = fib_rna.obsp['connectivities']

scv.tl.paga(fib_rna, groups='leiden_sub')

In [ ]:
scv.pl.paga(fib_rna, basis='umap', size=50, alpha=.1,
            legend_fontoutline=2.5, legend_loc = 'right',
            min_edge_width=2, node_size_scale=1.5,)

## Wound healing times as partitions

In [ ]:
scv.pl.velocity_embedding_stream(fib_rna, basis='umap', color='sample', legend_fontoutline=2.5, legend_loc = 'right')

In [ ]:
# this is needed due to a current bug - bugfix is coming soon.
fib_rna.uns['neighbors']['distances'] = fib_rna.obsp['distances']
fib_rna.uns['neighbors']['connectivities'] = fib_rna.obsp['connectivities']

scv.tl.paga(fib_rna, groups='sample')

In [ ]:
scv.pl.paga(fib_rna, basis='umap', size=50, alpha=.1,
            legend_fontoutline=2.5, legend_loc = 'right',
            min_edge_width=2, node_size_scale=1.5,)

# PAGA

## Subclusters as partitions

In [ ]:
sc.tl.paga(fibroblasts_merged, groups='leiden_sub')

In [ ]:
sc.pl.paga(fibroblasts_merged, color='leiden_sub')

## Wound healing times as partitions

In [ ]:
sc.tl.paga(fibroblasts_merged, groups='sample')

In [ ]:
sc.pl.paga(fibroblasts_merged, color='sample')